In [1]:
import os, glob, gc, rasterio,joblib, numpy as np, geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.io import MemoryFile
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy.ndimage import uniform_filter

In [5]:
North_image_safe_path=r'E:\GIS\Smart Harbour\Final_Tool_prod\S2A_MSIL1C_20230922T154951_N0510_R054_T18TXS_20241027T033244.SAFE'
South_image_safe_path=r'E:\GIS\Smart Harbour\Final_Tool_prod\S2A_MSIL1C_20230922T154951_N0509_R054_T18TXR_20230922T211005.SAFE'
STL_river_path=r'E:\GIS\Smart Harbour\Final_Tool_prod\Study_area_shapefile\StLawrence_Bathy_Buffer_10km_26918.shp'
RF_model_path='E:\GIS\Smart Harbour\Final_Tool_prod\RFbathy.joblib'
output_path=r'E:\GIS\Smart Harbour\Final_Tool_prod' #where you want the output files
mask_path=r'E:\GIS\Smart Harbour\Final_Tool_prod\Osd_StL_c26918.tif' #all Nodata pixels in this raster are masked

#LOCAL VARIABLES
mosaic_path=f'{output_path}\\STL_5band_c.tif'
image_with_filters_path=f'{output_path}\\STL_5band_c_filter.tif'
bathy_path=f'{output_path}\\bathySTL_2023-0922.tif'
bathy_path_25=f'{output_path}\\bathySTL25m_2023-0922.tif'

In [6]:
def get_band_paths(safe_folder, bands=['B01', 'B02', 'B03', 'B04', 'B05']):
    paths = {b: (glob.glob(os.path.join(safe_folder, '**', f'*{b}.jp2'), recursive=True) or [None])[0]for b in bands}
    if None in paths.values():
        print(f"Missing bands: {[b for b, p in paths.items() if p is None]} in {safe_folder}")
    return paths
def resample_band(src_path, ref_raster, resampling=Resampling.bilinear):
    """Resample a band to match the reference raster in resolution and extent."""
    with rasterio.open(src_path) as src:
        transform, w, h = calculate_default_transform(src.crs, ref_raster.crs, ref_raster.width,
                                                      ref_raster.height, *ref_raster.bounds)
        resampled = np.empty((h, w), dtype=np.uint16)
        reproject(source=src.read(1),destination=resampled,src_transform=src.transform,src_crs=src.crs,
            dst_transform=transform,dst_crs=ref_raster.crs,resampling=resampling)
    return resampled
def create_composite_in_memory(safe_folder):
    band_paths = get_band_paths(safe_folder)
    with rasterio.open(band_paths['B02']) as b02:
        profile = b02.profile.copy()
        profile.update(count=5, dtype='uint16', driver="GTiff")
        stacked = np.stack([
            resample_band(band_paths[b], b02) if b in ['B01', 'B05'] else rasterio.open(p).read(1).astype(np.uint16)
            for b, p in band_paths.items()])
        memfile = MemoryFile()
        dataset = memfile.open(**profile)
        for i in range(5):
            dataset.write(stacked[i], i + 1)
    return dataset
def mosaic_and_clip(raster_datasets, shapefile_path, output_path):
    if not raster_datasets:
        raise ValueError("No raster data provided for mosaicking.")
    mosaic, transform = merge(raster_datasets)
    meta = raster_datasets[0].meta.copy()
    meta.update({"height": mosaic.shape[1], "width": mosaic.shape[2], "transform": transform})
    if shapefile_path:
        print(f"Clipping mosaic to {shapefile_path}")
        shapes = [f["geometry"] for f in gpd.read_file(shapefile_path).__geo_interface__["features"]]
        with rasterio.MemoryFile() as memfile:
            with memfile.open(**meta) as raster:  # Removed `driver="GTiff"`
                raster.write(mosaic)
                clipped, clipped_transform = mask(raster, shapes, crop=True)
        meta.update({"height": clipped.shape[1], "width": clipped.shape[2], "transform": clipped_transform})
        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(clipped)
        print(f"Clipped mosaic saved: {output_path}")
    else:
        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(mosaic)
        print(f"Mosaic saved: {output_path}")
    return output_path
def process_north_south_mosaic(north_safe, south_safe, shapefile):
    north_dataset = create_composite_in_memory(north_safe)
    south_dataset = create_composite_in_memory(south_safe)
    return mosaic_and_clip([north_dataset, south_dataset], shapefile, mosaic_path)
def apply_filter(image_band, kernel_size):
    return uniform_filter(image_band, size=kernel_size, mode='mirror')
def process_and_save_filtered_image(input_image_path):
    print(f'Making filtered image using {input_image_path}')
    kernel_sizes = [3, 5, 7, 9, 11,13,15]
    with rasterio.open(input_image_path) as src:
        img = src.read()
        profile = src.profile
        profile.update(count=img.shape[0] + len(kernel_sizes) * img.shape[0])# Update count for new bands (input bands + filtered bands)
        # Create a list to hold all bands (original + filtered)
        output_bands = [img[i] for i in range(img.shape[0])]  # Start with original bands
        for band_index in range(img.shape[0]):
            for kernel_size in kernel_sizes:
                filtered_band = apply_filter(img[band_index].astype(np.float32), kernel_size).astype(np.uint16)
                output_bands.append(filtered_band)
                del filtered_band
                gc.collect()
            print(f'Filters made for b{band_index+1}')
        output_image = np.stack(output_bands, axis=0).astype(np.uint16)
        print(f" Filter image shape: {output_image.shape}")
        base, ext = os.path.splitext(input_image_path)  # Split the input file into name and extension
        output_file_path = f"{base}_filter{ext}"
        with rasterio.open(output_file_path, 'w', **profile) as dst:
            dst.write(output_image)
        del output_image,output_bands
        gc.collect()
def predict_water_depth(image_with_filters_path, mask_path, rf_model_path, output_path, nodata_value=-100):
    rf_model = joblib.load(rf_model_path)
    with rasterio.open(image_with_filters_path) as src:
        img = src.read()
        profile = src.profile
    with rasterio.open(mask_path) as mask_src:
        osd_mask = mask_src.read(1)  # Assume single-band mask
    valid_mask = (osd_mask != mask_src.nodata)# Identify valid water pixels (where OSD mask is not NoData)
    water_pixel_indices = np.where(valid_mask.ravel())
    feature_stack = np.column_stack([band.ravel() for band in img])# Extract features for prediction
    water_pixel_features = feature_stack[water_pixel_indices]
    print(f"Number of water pixels: {len(water_pixel_indices[0])}\nStarting RF prediciton")
    predictions = rf_model.predict(water_pixel_features)
    del water_pixel_features,feature_stack
    gc.collect()
    predicted_depths = np.full((img.shape[1], img.shape[2]), nodata_value, dtype=np.float32)
    predicted_depths.ravel()[water_pixel_indices] = predictions
    profile.update(dtype=rasterio.float32, count=1, nodata=nodata_value,compress='zstd',predictor=2)
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(predicted_depths, 1)
    print(f"Saved predicted depths to {output_path}")
def mask_raster(input_tif, output_tif, threshold):
    with rasterio.open(input_tif) as src:
        data = src.read(1)
        dtype = data.dtype 
        mask = data > threshold
        data[mask] = -100
        meta = src.meta.copy()
        meta.update({'nodata': -100,'dtype': dtype,"compress":'zstd',"predictor":2})
    with rasterio.open(output_tif, 'w', **meta) as dst:
        dst.write(data, 1)  # Write the modified data to the first band
    print(f"Saved masked depths to {output_tif}")

In [7]:
process_north_south_mosaic(North_image_safe_path, South_image_safe_path, STL_river_path)
process_and_save_filtered_image(mosaic_path)
predict_water_depth(image_with_filters_path, mask_path, RF_model_path, bathy_path, nodata_value=-100)
mask_raster(bathy_path, bathy_path_25, 2.5)

Clipping mosaic to E:\GIS\Smart Harbour\Final_Tool_prod\Study_area_shapefile\StLawrence_Bathy_Buffer_10km_26918.shp
Clipped mosaic saved: E:\GIS\Smart Harbour\Final_Tool_prod\STL_5band_c.tif
Making filtered image using E:\GIS\Smart Harbour\Final_Tool_prod\STL_5band_c.tif
Filters made for b1
Filters made for b2
Filters made for b3
Filters made for b4
Filters made for b5
 Filter image shape: (40, 11957, 9794)
Number of water pixels: 5516359
Starting RF prediciton
Saved predicted depths to E:\GIS\Smart Harbour\Final_Tool_prod\bathySTL_2023-0922.tif
Saved masked depths to E:\GIS\Smart Harbour\Final_Tool_prod\bathySTL25m_2023-0922.tif
